# Instructions
- Make conda environment and install libraries
- Install Stockfish and input path to it
- Input path to dataset
- Run cells

In [1]:
# standard lib
import io

# data analysis
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# utils
from tqdm.notebook import tqdm

# chess
import chess
import chess
import chess.pgn
import chess.engine

# my custom libs
from gameeval import landscapeeval, gameeval
from mmaformatter import save_as_mma

In [2]:
df = pd.read_csv("../data/resampled_100000_games.csv")

In [3]:
df["Event"].value_counts()

Event
Rated Rapid game     33335
Rated Blitz game     33334
Rated Bullet game    33331
Name: count, dtype: int64

In [4]:
df["Termination"].value_counts()

Termination
Normal          69544
Time forfeit    30456
Name: count, dtype: int64

In [5]:
# Make evaluations of position using stockfish
# Path to stockfish engine
# engine = chess.engine.SimpleEngine.popen_uci("/usr/local/Cellar/stockfish/17/bin/stockfish")
engine_path = "/usr/local/Cellar/stockfish/17/bin/stockfish"

In [6]:
# analyze all 100,000 games (just mainline moves and k=2 tree)
cp_evals = []
wdl_evals = []
landscapes = []
k = 2
d = 1
depth = 15
for moves in tqdm(df["Moves"]):
    pgn = io.StringIO(moves)
    game = chess.pgn.read_game(pgn)
    cps, wdls = await gameeval(game, engine_path, depth=depth)
    landscape = await landscapeeval(game, engine_path, k=k, d=d, depth=depth)
    cp_evals.append(cps)
    wdl_evals.append(wdls)
    landscapes.append(landscape)

  0%|          | 0/2 [00:00<?, ?it/s]

In [9]:
15*100000/3600

416.6666666666667

In [ ]:
# save results as Mathematica files
save_as_mma([c.tolist() for c in cp_evals], "../Centipawns.mx")
save_as_mma([w.tolist() for w in wdl_evals], "../WDLs.mx")
save_as_mma(landscapes, "../Landscapes_k=2_d=1.mx")